## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.04,78,96,8.12,overcast clouds
1,1,Vestmannaeyjar,IS,63.4427,-20.2734,52.79,82,40,12.66,scattered clouds
2,2,Cap Malheureux,MU,-19.9842,57.6142,71.47,69,5,5.99,clear sky
3,3,Iberia,US,29.9502,-91.7507,82.98,89,1,4.61,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,45.81,87,100,28.36,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = int(input("Please enter your desired maximum temperature "))
min_temp = int(input("Please enter your desired minimum temperature "))

Please enter your desired maximum temperature 90
Please enter your desired minimum temperature 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
62,62,Rincon,US,32.2960,-81.2354,88.25,71,75,9.22,broken clouds
74,74,Cayenne,GF,4.9333,-52.3333,89.64,77,75,4.00,broken clouds
114,114,Kasempa,ZM,-13.4584,25.8338,89.64,11,11,6.17,few clouds
124,124,Camocim,BR,-2.9022,-40.8411,89.58,51,50,12.17,scattered clouds
200,200,Champerico,GT,14.3000,-91.9167,88.75,77,98,2.51,overcast clouds
247,247,Santa Isabel Do Rio Negro,BR,-0.4139,-65.0192,89.06,56,90,1.92,overcast clouds
264,264,Bodden Town,KY,19.2833,-81.2500,89.60,70,20,7.00,few clouds
279,279,Abeche,TD,13.8292,20.8324,89.55,45,81,10.71,broken clouds
316,316,Kutum,SD,14.2000,24.6667,89.01,28,100,12.88,overcast clouds
371,371,Garowe,SO,8.4054,48.4845,88.97,37,99,22.08,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                21
City                   21
Country                21
Lat                    21
Lng                    21
Max Temp               21
Humidity               21
Cloudiness             21
Wind Speed             21
Current Description    21
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
62,Rincon,US,88.25,broken clouds,32.2960,-81.2354,
74,Cayenne,GF,89.64,broken clouds,4.9333,-52.3333,
114,Kasempa,ZM,89.64,few clouds,-13.4584,25.8338,
124,Camocim,BR,89.58,scattered clouds,-2.9022,-40.8411,
200,Champerico,GT,88.75,overcast clouds,14.3000,-91.9167,
247,Santa Isabel Do Rio Negro,BR,89.06,overcast clouds,-0.4139,-65.0192,
264,Bodden Town,KY,89.60,few clouds,19.2833,-81.2500,
279,Abeche,TD,89.55,broken clouds,13.8292,20.8324,
316,Kutum,SD,89.01,overcast clouds,14.2000,24.6667,
371,Garowe,SO,88.97,overcast clouds,8.4054,48.4845,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    # If an error is experienced, skip the hotel.
    except (IndexError):
        print("Hotel not found... skipping.")
        pass


Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
62,Rincon,US,88.25,broken clouds,32.2960,-81.2354,Days Inn Rincon
74,Cayenne,GF,89.64,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
114,Kasempa,ZM,89.64,few clouds,-13.4584,25.8338,Lolelunga Forest Reserve
124,Camocim,BR,89.58,scattered clouds,-2.9022,-40.8411,Hotel Ilha do Amor
200,Champerico,GT,88.75,overcast clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
247,Santa Isabel Do Rio Negro,BR,89.06,overcast clouds,-0.4139,-65.0192,Hotel GSA
264,Bodden Town,KY,89.60,few clouds,19.2833,-81.2500,Will T Place
279,Abeche,TD,89.55,broken clouds,13.8292,20.8324,Assaleh Hamdan Haroune
316,Kutum,SD,89.01,overcast clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
371,Garowe,SO,88.97,overcast clouds,8.4054,48.4845,Curubo Hotel


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))